<a href="https://colab.research.google.com/github/fralfaro/r4ds-book/blob/main/docs/13_relational_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relational data

## Introduction

It's rare that a data analysis involves only a single table of data.
Typically you have many tables of data, and you must combine them to answer the questions that you're interested in.
Collectively, multiple tables of data are called **relational data** because it is the relations, not just the individual datasets, that are important.

Relations are always defined between a pair of tables.
All other relations are built up from this simple idea: the relations of three or more tables are always a property of the relations between each pair.
Sometimes both elements of a pair can be the same table!
This is needed if, for example, you have a table of people, and each person has a reference to their parents.

To work with relational data you need verbs that work with pairs of tables.
There are three families of verbs designed to work with relational data:

-   **Mutating joins**, which add new variables to one data frame from matching observations in another.

-   **Filtering joins**, which filter observations from one data frame based on whether or not they match an observation in the other table.

-   **Set operations**, which treat observations as if they were set elements.

The most common place to find relational data is in a *relational* database management system (or RDBMS), a term that encompasses almost all modern databases.
If you've used a database before, you've almost certainly used SQL.
If so, you should find the concepts in this chapter familiar, although their expression in dplyr is a little different.
Generally, dplyr is a little easier to use than SQL because dplyr is specialised to do data analysis: it makes common data analysis operations easier, at the expense of making it more difficult to do other things that aren't commonly needed for data analysis.

### Prerequisites

We will explore relational data from `nycflights13` using the two-table verbs from dplyr.


In [1]:
install.packages("nycflights13")
library(tidyverse)
library(nycflights13)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()




## nycflights13 

We will use the nycflights13 package to learn about relational data.
nycflights13 contains five tibbles : `airlines`, `airports`, `weather` and `planes` which are all related to the `flights` table that you used in [data transformation]:




-   `airlines` lets you look up the full carrier name from its abbreviated code:


In [2]:
airlines

carrier,name
<chr>,<chr>
9E,Endeavor Air Inc.
AA,American Airlines Inc.
AS,Alaska Airlines Inc.
B6,JetBlue Airways
DL,Delta Air Lines Inc.
EV,ExpressJet Airlines Inc.
F9,Frontier Airlines Inc.
FL,AirTran Airways Corporation
HA,Hawaiian Airlines Inc.



-   `airports` gives information about each airport, identified by the `faa` airport code:


In [3]:
airports

faa,name,lat,lon,alt,tz,dst,tzone
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
04G,Lansdowne Airport,41.13047,-80.61958,1044,-5,A,America/New_York
06A,Moton Field Municipal Airport,32.46057,-85.68003,264,-6,A,America/Chicago
06C,Schaumburg Regional,41.98934,-88.10124,801,-6,A,America/Chicago
06N,Randall Airport,41.43191,-74.39156,523,-5,A,America/New_York
09J,Jekyll Island Airport,31.07447,-81.42778,11,-5,A,America/New_York
0A9,Elizabethton Municipal Airport,36.37122,-82.17342,1593,-5,A,America/New_York
0G6,Williams County Airport,41.46731,-84.50678,730,-5,A,America/New_York
0G7,Finger Lakes Regional Airport,42.88356,-76.78123,492,-5,A,America/New_York
0P2,Shoestring Aviation Airfield,39.79482,-76.64719,1000,-5,U,America/New_York



-   `planes` gives information about each plane, identified by its `tailnum`:


In [4]:
planes

tailnum,year,type,manufacturer,model,engines,seats,speed,engine
<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>
N10156,2004,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NA,Turbo-fan
N102UW,1998,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N103US,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N104UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N10575,2002,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NA,Turbo-fan
N105UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N107US,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N108UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan
N109UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NA,Turbo-fan



-   `weather` gives the weather at each NYC airport for each hour:


In [5]:
weather

origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
EWR,2013,1,1,1,39.02,26.06,59.37,270,10.35702,NA,0,1012.0,10,2013-01-01 01:00:00
EWR,2013,1,1,2,39.02,26.96,61.63,250,8.05546,NA,0,1012.3,10,2013-01-01 02:00:00
EWR,2013,1,1,3,39.02,28.04,64.43,240,11.50780,NA,0,1012.5,10,2013-01-01 03:00:00
EWR,2013,1,1,4,39.92,28.04,62.21,250,12.65858,NA,0,1012.2,10,2013-01-01 04:00:00
EWR,2013,1,1,5,39.02,28.04,64.43,260,12.65858,NA,0,1011.9,10,2013-01-01 05:00:00
EWR,2013,1,1,6,37.94,28.04,67.21,240,11.50780,NA,0,1012.4,10,2013-01-01 06:00:00
EWR,2013,1,1,7,39.02,28.04,64.43,240,14.96014,NA,0,1012.2,10,2013-01-01 07:00:00
EWR,2013,1,1,8,39.92,28.04,62.21,250,10.35702,NA,0,1012.2,10,2013-01-01 08:00:00
EWR,2013,1,1,9,39.92,28.04,62.21,260,14.96014,NA,0,1012.7,10,2013-01-01 09:00:00



One way to show the relationships between the different tables is with a drawing:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/245292d1ea724f6c3fd8a92063dcd7bfb9758d02/5751b/diagrams/relational-nycflights.png" alt="img R" >
</center>


This diagram is a little overwhelming, but it's simple compared to some you'll see in the wild!
The key to understanding diagrams like this is to remember each relation always concerns a pair of tables.
You don't need to understand the whole thing; you just need to understand the chain of relations between the tables that you are interested in.

For nycflights13:

-   `flights` connects to `planes` via a single variable, `tailnum`.

-   `flights` connects to `airlines` through the `carrier` variable.

-   `flights` connects to `airports` in two ways: via the `origin` and `dest` variables.

-   `flights` connects to `weather` via `origin` (the location), and `year`, `month`, `day` and `hour` (the time).

### Exercises

1.  Imagine you wanted to draw (approximately) the route each plane flies from its origin to its destination.
    What variables would you need?
    What tables would you need to combine?

2.  I forgot to draw the relationship between `weather` and `airports`.
    What is the relationship and how should it appear in the diagram?

3.  `weather` only contains information for the origin (NYC) airports.
    If it contained weather records for all airports in the USA, what additional relation would it define with `flights`?

## Keys

The variables used to connect each pair of tables are called **keys**.
A key is a variable (or set of variables) that uniquely identifies an observation.
In simple cases, a single variable is sufficient to identify an observation.
For example, each plane is uniquely identified by its `tailnum`.
In other cases, multiple variables may be needed.
For example, to identify an observation in `weather` you need five variables: `year`, `month`, `day`, `hour`, and `origin`.

There are two types of keys:

-   A **primary key** uniquely identifies an observation in its own table.
    For example, `planes$tailnum` is a primary key because it uniquely identifies each plane in the `planes` table.

-   A **foreign key** uniquely identifies an observation in another table.
    For example, `flights$tailnum` is a foreign key because it appears in the `flights` table where it matches each flight to a unique plane.

A variable can be both a primary key *and* a foreign key.
For example, `origin` is part of the `weather` primary key, and is also a foreign key for the `airports` table.

Once you've identified the primary keys in your tables, it's good practice to verify that they do indeed uniquely identify each observation.
One way to do that is to `count()` the primary keys and look for entries where `n` is greater than one:


In [6]:
planes %>% 
  count(tailnum) %>% 
  filter(n > 1)

tailnum,n
<chr>,<int>


In [7]:
weather %>% 
  count(year, month, day, hour, origin) %>% 
  filter(n > 1)

year,month,day,hour,origin,n
<int>,<int>,<int>,<int>,<chr>,<int>
2013,11,3,1,EWR,2
2013,11,3,1,JFK,2
2013,11,3,1,LGA,2



Sometimes a table doesn't have an explicit primary key: each row is an observation, but no combination of variables reliably identifies it.
For example, what's the primary key in the `flights` table?
You might think it would be the date plus the flight or tail number, but neither of those are unique:


In [8]:
flights %>% 
  count(year, month, day, flight) %>% 
  filter(n > 1)

year,month,day,flight,n
<int>,<int>,<int>,<int>,<int>
2013,1,1,1,2
2013,1,1,3,2
2013,1,1,4,2
2013,1,1,11,3
2013,1,1,15,2
2013,1,1,21,2
2013,1,1,27,4
2013,1,1,31,2
2013,1,1,32,2


In [9]:
flights %>% 
  count(year, month, day, tailnum) %>% 
  filter(n > 1)

year,month,day,tailnum,n
<int>,<int>,<int>,<chr>,<int>
2013,1,1,N0EGMQ,2
2013,1,1,N11189,2
2013,1,1,N11536,2
2013,1,1,N11544,3
2013,1,1,N11551,2
2013,1,1,N12540,2
2013,1,1,N12567,2
2013,1,1,N13123,2
2013,1,1,N13538,3




When starting to work with this data, I had naively assumed that each flight number would be only used once per day: that would make it much easier to communicate problems with a specific flight.
Unfortunately that is not the case!
If a table lacks a primary key, it's sometimes useful to add one with `mutate()` and `row_number()`.
That makes it easier to match observations if you've done some filtering and want to check back in with the original data.
This is called a **surrogate key**.

A primary key and the corresponding foreign key in another table form a **relation**.
Relations are typically one-to-many.
For example, each flight has one plane, but each plane has many flights.
In other data, you'll occasionally see a 1-to-1 relationship.
You can think of this as a special case of 1-to-many.
You can model many-to-many relations with a many-to-1 relation plus a 1-to-many relation.
For example, in this data there's a many-to-many relationship between airlines and airports: each airline flies to many airports; each airport hosts many airlines.

### Exercises

1.  Add a surrogate key to `flights`.

2.  We know that some days of the year are "special", and fewer people than usual fly on them.
    How might you represent that data as a data frame?
    What would be the primary keys of that table?
    How would it connect to the existing tables?

3.  Identify the keys in the following datasets

    a.  `Lahman::Batting`,
    b.  `babynames::babynames`
    c.  `nasaweather::atmos`
    d.  `fueleconomy::vehicles`
    e.  `ggplot2::diamonds`

    (You might need to install some packages and read some documentation.)

4.  Draw a diagram illustrating the connections between the `Batting`, `People`, and `Salaries` tables in the Lahman package.
    Draw another diagram that shows the relationship between `People`, `Managers`, `AwardsManagers`.

    How would you characterise the relationship between the `Batting`, `Pitching`, and `Fielding` tables?

## Mutating joins 

The first tool we'll look at for combining a pair of tables is the **mutating join**.
A mutating join allows you to combine variables from two tables.
It first matches observations by their keys, then copies across variables from one table to the other.

Like `mutate()`, the join functions add variables to the right, so if you have a lot of variables already, the new variables won't get printed out.
For these examples, we'll make it easier to see what's going on in the examples by creating a narrower dataset:


In [10]:
flights2 <- flights %>% 
  select(year:day, hour, origin, dest, tailnum, carrier)
flights2

year,month,day,hour,origin,dest,tailnum,carrier
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>
2013,1,1,5,EWR,IAH,N14228,UA
2013,1,1,5,LGA,IAH,N24211,UA
2013,1,1,5,JFK,MIA,N619AA,AA
2013,1,1,5,JFK,BQN,N804JB,B6
2013,1,1,6,LGA,ATL,N668DN,DL
2013,1,1,5,EWR,ORD,N39463,UA
2013,1,1,6,EWR,FLL,N516JB,B6
2013,1,1,6,LGA,IAD,N829AS,EV
2013,1,1,6,JFK,MCO,N593JB,B6



(Remember, when you're in RStudio, you can also use `View()` to avoid this problem.)

Imagine you want to add the full airline name to the `flights2` data.
You can combine the `airlines` and `flights2` data frames with `left_join()`:


In [11]:
flights2 %>%
  select(-origin, -dest) %>% 
  left_join(airlines, by = "carrier")

year,month,day,hour,tailnum,carrier,name
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
2013,1,1,5,N14228,UA,United Air Lines Inc.
2013,1,1,5,N24211,UA,United Air Lines Inc.
2013,1,1,5,N619AA,AA,American Airlines Inc.
2013,1,1,5,N804JB,B6,JetBlue Airways
2013,1,1,6,N668DN,DL,Delta Air Lines Inc.
2013,1,1,5,N39463,UA,United Air Lines Inc.
2013,1,1,6,N516JB,B6,JetBlue Airways
2013,1,1,6,N829AS,EV,ExpressJet Airlines Inc.
2013,1,1,6,N593JB,B6,JetBlue Airways



The result of joining airlines to flights2 is an additional variable: `name`.
This is why I call this type of join a mutating join.
In this case, you could have got to the same place using `mutate()` and R's base subsetting:


In [12]:
flights2 %>%
  select(-origin, -dest) %>% 
  mutate(name = airlines$name[match(carrier, airlines$carrier)])

year,month,day,hour,tailnum,carrier,name
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
2013,1,1,5,N14228,UA,United Air Lines Inc.
2013,1,1,5,N24211,UA,United Air Lines Inc.
2013,1,1,5,N619AA,AA,American Airlines Inc.
2013,1,1,5,N804JB,B6,JetBlue Airways
2013,1,1,6,N668DN,DL,Delta Air Lines Inc.
2013,1,1,5,N39463,UA,United Air Lines Inc.
2013,1,1,6,N516JB,B6,JetBlue Airways
2013,1,1,6,N829AS,EV,ExpressJet Airlines Inc.
2013,1,1,6,N593JB,B6,JetBlue Airways



But this is hard to generalise when you need to match multiple variables, and takes close reading to figure out the overall intent.

The following sections explain, in detail, how mutating joins work.
You'll start by learning a useful visual representation of joins.
We'll then use that to explain the four mutating join functions: the inner join, and the three outer joins.
When working with real data, keys don't always uniquely identify observations, so next we'll talk about what happens when there isn't a unique match.
Finally, you'll learn how to tell dplyr which variables are the keys for a given join.

### Understanding joins

To help you learn how joins work, I'm going to use a visual representation:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/108c0749d084c03103f8e1e8276c20e06357b124/5f113/diagrams/join-setup.png" alt="img R" >
</center>

In [13]:
x <- tribble(
  ~key, ~val_x,
     1, "x1",
     2, "x2",
     3, "x3"
)
y <- tribble(
  ~key, ~val_y,
     1, "y1",
     2, "y2",
     4, "y3"
)


The coloured column represents the "key" variable: these are used to match the rows between the tables.
The grey column represents the "value" column that is carried along for the ride.
In these examples I'll show a single key variable, but the idea generalises in a straightforward way to multiple keys and multiple values.

A join is a way of connecting each row in `x` to zero, one, or more rows in `y`.
The following diagram shows each potential match as an intersection of a pair of lines.


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/820b012580731f2134f90ee9c6388994c2343683/27703/diagrams/join-setup2.png" alt="img R" >
</center>


(If you look closely, you might notice that we've switched the order of the key and value columns in `x`. This is to emphasise that joins match based on the key; the value is just carried along for the ride.)

In an actual join, matches will be indicated with dots.
The number of dots = the number of matches = the number of rows in the output.


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png" alt="img R" >
</center>



### Inner join 
The simplest type of join is the **inner join**.
An inner join matches pairs of observations whenever their keys are equal:



<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png" alt="img R" >
</center>



(To be precise, this is an inner **equijoin** because the keys are matched using the equality operator. Since most joins are equijoins we usually drop that specification.)

The output of an inner join is a new data frame that contains the key, the x values, and the y values.
We use `by` to tell dplyr which variable is the key:



In [14]:
x %>% 
  inner_join(y, by = "key")

key,val_x,val_y
<dbl>,<chr>,<chr>
1,x1,y1
2,x2,y2



The most important property of an inner join is that unmatched rows are not included in the result.
This means that generally inner joins are usually not appropriate for use in analysis because it's too easy to lose observations.

### Outer joins 

An inner join keeps observations that appear in both tables.
An **outer join** keeps observations that appear in at least one of the tables.
There are three types of outer joins:

-   A **left join** keeps all observations in `x`.
-   A **right join** keeps all observations in `y`.
-   A **full join** keeps all observations in `x` and `y`.

These joins work by adding an additional "virtual" observation to each table.
This observation has a key that always matches (if no other key matches), and a value filled with `NA`.

Graphically, that looks like:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/9c12ca9e12ed26a7c5d2aa08e36d2ac4fb593f1e/79980/diagrams/join-outer.png" alt="img R" >
</center>


The most commonly used join is the left join: you use this whenever you look up additional data from another table, because it preserves the original observations even when there isn't a match.
The left join should be your default join: use it unless you have a strong reason to prefer one of the others.

Another way to depict the different types of joins is with a Venn diagram:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/aeab386461820b029b7e7606ccff1286f623bae1/ef0d4/diagrams/join-venn.png" alt="img R" >
</center>


However, this is not a great representation.
It might jog your memory about which join preserves the observations in which table, but it suffers from a major limitation: a Venn diagram can't show what happens when keys don't uniquely identify an observation.

### Duplicate keys 

So far all the diagrams have assumed that the keys are unique.
But that's not always the case.
This section explains what happens when the keys are not unique.
There are two possibilities:

1.  One table has duplicate keys.
    This is useful when you want to add in additional information as there is typically a one-to-many relationship.


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/6faac3e996263827cb57fc5803df6192541a9a4b/c7d74/diagrams/join-one-to-many.png" alt="img R" >
</center>



  Note that I've put the key column in a slightly different position in the output.
    This reflects that the key is a primary key in `y` and a foreign key in `x`.


In [15]:
x <- tribble(
  ~key, ~val_x,
     1, "x1",
     2, "x2",
     2, "x3",
     1, "x4"
)
y <- tribble(
  ~key, ~val_y,
     1, "y1",
     2, "y2"
)
left_join(x, y, by = "key")

key,val_x,val_y
<dbl>,<chr>,<chr>
1,x1,y1
2,x2,y2
2,x3,y2
1,x4,y1



2.  Both tables have duplicate keys.
    This is usually an error because in neither table do the keys uniquely identify an observation.
    When you join duplicated keys, you get all possible combinations, the Cartesian product:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/d37530bbf7749f48c02684013ae72b2996b07e25/37510/diagrams/join-many-to-many.png" alt="img R" >
</center>

In [16]:
x <- tribble(
  ~key, ~val_x,
     1, "x1",
     2, "x2",
     2, "x3",
     3, "x4"
)
y <- tribble(
  ~key, ~val_y,
     1, "y1",
     2, "y2",
     2, "y3",
     3, "y4"
)
left_join(x, y, by = "key")

key,val_x,val_y
<dbl>,<chr>,<chr>
1,x1,y1
2,x2,y2
2,x2,y3
2,x3,y2
2,x3,y3
3,x4,y4



### Defining the key columns 

So far, the pairs of tables have always been joined by a single variable, and that variable has the same name in both tables.
That constraint was encoded by `by = "key"`.
You can use other values for `by` to connect the tables in other ways:

-   The default, `by = NULL`, uses all variables that appear in both tables, the so called **natural** join.
    For example, the flights and weather tables match on their common variables: `year`, `month`, `day`, `hour` and `origin`.


In [17]:
flights2 %>% 
  left_join(weather)

Joining, by = c("year", "month", "day", "hour", "origin")


year,month,day,hour,origin,dest,tailnum,carrier,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,5,EWR,IAH,N14228,UA,39.02,28.04,64.43,260,12.65858,NA,0,1011.9,10,2013-01-01 05:00:00
2013,1,1,5,LGA,IAH,N24211,UA,39.92,24.98,54.81,250,14.96014,21.86482,0,1011.4,10,2013-01-01 05:00:00
2013,1,1,5,JFK,MIA,N619AA,AA,39.02,26.96,61.63,260,14.96014,NA,0,1012.1,10,2013-01-01 05:00:00
2013,1,1,5,JFK,BQN,N804JB,B6,39.02,26.96,61.63,260,14.96014,NA,0,1012.1,10,2013-01-01 05:00:00
2013,1,1,6,LGA,ATL,N668DN,DL,39.92,24.98,54.81,260,16.11092,23.01560,0,1011.7,10,2013-01-01 06:00:00
2013,1,1,5,EWR,ORD,N39463,UA,39.02,28.04,64.43,260,12.65858,NA,0,1011.9,10,2013-01-01 05:00:00
2013,1,1,6,EWR,FLL,N516JB,B6,37.94,28.04,67.21,240,11.50780,NA,0,1012.4,10,2013-01-01 06:00:00
2013,1,1,6,LGA,IAD,N829AS,EV,39.92,24.98,54.81,260,16.11092,23.01560,0,1011.7,10,2013-01-01 06:00:00
2013,1,1,6,JFK,MCO,N593JB,B6,37.94,26.96,64.29,260,13.80936,NA,0,1012.6,10,2013-01-01 06:00:00



-   A character vector, `by = "x"`.
    This is like a natural join, but uses only some of the common variables.
    For example, `flights` and `planes` have `year` variables, but they mean different things so we only want to join by `tailnum`.


In [18]:
flights2 %>% 
  left_join(planes, by = "tailnum")

year.x,month,day,hour,origin,dest,tailnum,carrier,year.y,type,manufacturer,model,engines,seats,speed,engine
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>
2013,1,1,5,EWR,IAH,N14228,UA,1999,Fixed wing multi engine,BOEING,737-824,2,149,NA,Turbo-fan
2013,1,1,5,LGA,IAH,N24211,UA,1998,Fixed wing multi engine,BOEING,737-824,2,149,NA,Turbo-fan
2013,1,1,5,JFK,MIA,N619AA,AA,1990,Fixed wing multi engine,BOEING,757-223,2,178,NA,Turbo-fan
2013,1,1,5,JFK,BQN,N804JB,B6,2012,Fixed wing multi engine,AIRBUS,A320-232,2,200,NA,Turbo-fan
2013,1,1,6,LGA,ATL,N668DN,DL,1991,Fixed wing multi engine,BOEING,757-232,2,178,NA,Turbo-fan
2013,1,1,5,EWR,ORD,N39463,UA,2012,Fixed wing multi engine,BOEING,737-924ER,2,191,NA,Turbo-fan
2013,1,1,6,EWR,FLL,N516JB,B6,2000,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-232,2,200,NA,Turbo-fan
2013,1,1,6,LGA,IAD,N829AS,EV,1998,Fixed wing multi engine,CANADAIR,CL-600-2B19,2,55,NA,Turbo-fan
2013,1,1,6,JFK,MCO,N593JB,B6,2004,Fixed wing multi engine,AIRBUS,A320-232,2,200,NA,Turbo-fan



   Note that the `year` variables (which appear in both input data frames, but are not constrained to be equal) are disambiguated in the output with a suffix.

-   A named character vector: `by = c("a" = "b")`.
    This will match variable `a` in table `x` to variable `b` in table `y`.
    The variables from `x` will be used in the output.

    For example, if we want to draw a map we need to combine the flights data with the airports data which contains the location (`lat` and `lon`) of each airport.
    Each flight has an origin and destination `airport`, so we need to specify which one we want to join to:


In [20]:
flights2 %>% 
  left_join(airports, c("dest" = "faa"))

year,month,day,hour,origin,dest,tailnum,carrier,name,lat,lon,alt,tz,dst,tzone
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2013,1,1,5,EWR,IAH,N14228,UA,George Bush Intercontinental,29.98443,-95.34144,97,-6,A,America/Chicago
2013,1,1,5,LGA,IAH,N24211,UA,George Bush Intercontinental,29.98443,-95.34144,97,-6,A,America/Chicago
2013,1,1,5,JFK,MIA,N619AA,AA,Miami Intl,25.79325,-80.29056,8,-5,A,America/New_York
2013,1,1,5,JFK,BQN,N804JB,B6,NA,NA,NA,NA,NA,NA,NA
2013,1,1,6,LGA,ATL,N668DN,DL,Hartsfield Jackson Atlanta Intl,33.63672,-84.42807,1026,-5,A,America/New_York
2013,1,1,5,EWR,ORD,N39463,UA,Chicago Ohare Intl,41.97860,-87.90484,668,-6,A,America/Chicago
2013,1,1,6,EWR,FLL,N516JB,B6,Fort Lauderdale Hollywood Intl,26.07258,-80.15275,9,-5,A,America/New_York
2013,1,1,6,LGA,IAD,N829AS,EV,Washington Dulles Intl,38.94453,-77.45581,313,-5,A,America/New_York
2013,1,1,6,JFK,MCO,N593JB,B6,Orlando Intl,28.42939,-81.30899,96,-5,A,America/New_York


In [21]:

flights2 %>% 
  left_join(airports, c("origin" = "faa"))

year,month,day,hour,origin,dest,tailnum,carrier,name,lat,lon,alt,tz,dst,tzone
<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2013,1,1,5,EWR,IAH,N14228,UA,Newark Liberty Intl,40.69250,-74.16867,18,-5,A,America/New_York
2013,1,1,5,LGA,IAH,N24211,UA,La Guardia,40.77725,-73.87261,22,-5,A,America/New_York
2013,1,1,5,JFK,MIA,N619AA,AA,John F Kennedy Intl,40.63975,-73.77893,13,-5,A,America/New_York
2013,1,1,5,JFK,BQN,N804JB,B6,John F Kennedy Intl,40.63975,-73.77893,13,-5,A,America/New_York
2013,1,1,6,LGA,ATL,N668DN,DL,La Guardia,40.77725,-73.87261,22,-5,A,America/New_York
2013,1,1,5,EWR,ORD,N39463,UA,Newark Liberty Intl,40.69250,-74.16867,18,-5,A,America/New_York
2013,1,1,6,EWR,FLL,N516JB,B6,Newark Liberty Intl,40.69250,-74.16867,18,-5,A,America/New_York
2013,1,1,6,LGA,IAD,N829AS,EV,La Guardia,40.77725,-73.87261,22,-5,A,America/New_York
2013,1,1,6,JFK,MCO,N593JB,B6,John F Kennedy Intl,40.63975,-73.77893,13,-5,A,America/New_York



### Exercises

1.  Compute the average delay by destination, then join on the `airports` data frame so you can show the spatial distribution of delays.
    Here's an easy way to draw a map of the United States:

    ```r
    airports %>%
      semi_join(flights, c("faa" = "dest")) %>%
      ggplot(aes(lon, lat)) +
        borders("state") +
        geom_point() +
        coord_quickmap()
    ```

    (Don't worry if you don't understand what `semi_join()` does --- you'll learn about it next.)

    You might want to use the `size` or `colour` of the points to display the average delay for each airport.



2.  Add the location of the origin *and* destination (i.e. the `lat` and `lon`) to `flights`.

3.  Is there a relationship between the age of a plane and its delays?

4.  What weather conditions make it more likely to see a delay?

5.  What happened on June 13 2013?
    Display the spatial pattern of delays, and then use Google to cross-reference with the weather.



### Other implementations

`base::merge()` can perform all four types of mutating join:

| dplyr              | merge                                     |
|--------------------|-------------------------------------------|
| `inner_join(x, y)` | `merge(x, y)`                             |
| `left_join(x, y)`  | `merge(x, y, all.x = TRUE)`               |
| `right_join(x, y)` | `merge(x, y, all.y = TRUE)`,              |
| `full_join(x, y)`  | `merge(x, y, all.x = TRUE, all.y = TRUE)` |

The advantages of the specific dplyr verbs is that they more clearly convey the intent of your code: the difference between the joins is really important but concealed in the arguments of `merge()`.
dplyr's joins are considerably faster and don't mess with the order of the rows.

SQL is the inspiration for dplyr's conventions, so the translation is straightforward:

| dplyr                        | SQL                                            |
|------------------------------|------------------------------------------------|
| `inner_join(x, y, by = "z")` | `SELECT * FROM x INNER JOIN y USING (z)`       |
| `left_join(x, y, by = "z")`  | `SELECT * FROM x LEFT OUTER JOIN y USING (z)`  |
| `right_join(x, y, by = "z")` | `SELECT * FROM x RIGHT OUTER JOIN y USING (z)` |
| `full_join(x, y, by = "z")`  | `SELECT * FROM x FULL OUTER JOIN y USING (z)`  |

Note that "INNER" and "OUTER" are optional, and often omitted.

Joining different variables between the tables, e.g. `inner_join(x, y, by = c("a" = "b"))` uses a slightly different syntax in SQL: `SELECT * FROM x INNER JOIN y ON x.a = y.b`.
As this syntax suggests, SQL supports a wider range of join types than dplyr because you can connect the tables using constraints other than equality (sometimes called non-equijoins).

## Filtering joins 

Filtering joins match observations in the same way as mutating joins, but affect the observations, not the variables.
There are two types:

-   `semi_join(x, y)` **keeps** all observations in `x` that have a match in `y`.
-   `anti_join(x, y)` **drops** all observations in `x` that have a match in `y`.

Semi-joins are useful for matching filtered summary tables back to the original rows.
For example, imagine you've found the top ten most popular destinations:



In [22]:
top_dest <- flights %>%
  count(dest, sort = TRUE) %>%
  head(10)
top_dest

dest,n
<chr>,<int>
ORD,17283
ATL,17215
LAX,16174
BOS,15508
MCO,14082
CLT,14064
SFO,13331
FLL,12055
MIA,11728



Now you want to find each flight that went to one of those destinations.
You could construct a filter yourself:


In [23]:
flights %>% 
  filter(dest %in% top_dest$dest)

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,944,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,753,745,8,AA,301,N3ALAA,LGA,ORD,138,733,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,924,917,7,UA,194,N29129,JFK,LAX,345,2475,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,923,937,-14,UA,1124,N53441,EWR,SFO,361,2565,6,0,2013-01-01 06:00:00
2013,1,1,559,559,0,702,706,-4,B6,1806,N708JB,JFK,BOS,44,187,5,59,2013-01-01 05:00:00



But it's difficult to extend that approach to multiple variables.
For example, imagine that you'd found the 10 days with highest average delays.
How would you construct the filter statement that used `year`, `month`, and `day` to match it back to `flights`?

Instead you can use a semi-join, which connects the two tables like a mutating join, but instead of adding new columns, only keeps the rows in `x` that have a match in `y`:


In [24]:
flights %>% 
  semi_join(top_dest)

Joining, by = "dest"


year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>
2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 05:00:00
2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 06:00:00
2013,1,1,554,558,-4,740,728,12,UA,1696,N39463,EWR,ORD,150,719,5,58,2013-01-01 05:00:00
2013,1,1,555,600,-5,913,854,19,B6,507,N516JB,EWR,FLL,158,1065,6,0,2013-01-01 06:00:00
2013,1,1,557,600,-3,838,846,-8,B6,79,N593JB,JFK,MCO,140,944,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,753,745,8,AA,301,N3ALAA,LGA,ORD,138,733,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,924,917,7,UA,194,N29129,JFK,LAX,345,2475,6,0,2013-01-01 06:00:00
2013,1,1,558,600,-2,923,937,-14,UA,1124,N53441,EWR,SFO,361,2565,6,0,2013-01-01 06:00:00
2013,1,1,559,559,0,702,706,-4,B6,1806,N708JB,JFK,BOS,44,187,5,59,2013-01-01 05:00:00



Graphically, a semi-join looks like this:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/028065a7f353a932d70d2dfc82bc5c5966f768ad/85a30/diagrams/join-semi.png" alt="img R" 
</center>



Only the existence of a match is important; it doesn't matter which observation is matched.
This means that filtering joins never duplicate rows like mutating joins do:



<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/e1d0283160251afaeca35cba216736eb995fee00/1b3cd/diagrams/join-semi-many.png" alt="img R" 
</center>



The inverse of a semi-join is an anti-join.
An anti-join keeps the rows that *don't* have a match:


<center>
    <img src="https://d33wubrfki0l68.cloudfront.net/f29a85efd53a079cc84c14ba4ba6894e238c3759/c1408/diagrams/join-anti.png" alt="img R" 
</center>



Anti-joins are useful for diagnosing join mismatches.
For example, when connecting `flights` and `planes`, you might be interested to know that there are many `flights` that don't have a match in `planes`:


In [25]:
flights %>%
  anti_join(planes, by = "tailnum") %>%
  count(tailnum, sort = TRUE)

tailnum,n
<chr>,<int>
NA,2512
N725MQ,575
N722MQ,513
N723MQ,507
N713MQ,483
N735MQ,396
N0EGMQ,371
N534MQ,364
N542MQ,363



### Exercises

1.  What does it mean for a flight to have a missing `tailnum`?
    What do the tail numbers that don't have a matching record in `planes` have in common?
    (Hint: one variable explains \~90% of the problems.)

2.  Filter flights to only show flights with planes that have flown at least 100 flights.

3.  Combine `fueleconomy::vehicles` and `fueleconomy::common` to find only the records for the most common models.

4.  Find the 48 hours (over the course of the whole year) that have the worst delays.
    Cross-reference it with the `weather` data.
    Can you see any patterns?

5.  What does `anti_join(flights, airports, by = c("dest" = "faa"))` tell you?
    What does `anti_join(airports, flights, by = c("faa" = "dest"))` tell you?

6.  You might expect that there's an implicit relationship between plane and airline, because each plane is flown by a single airline.
    Confirm or reject this hypothesis using the tools you've learned above.

## Join problems

The data you've been working with in this chapter has been cleaned up so that you'll have as few problems as possible.
Your own data is unlikely to be so nice, so there are a few things that you should do with your own data to make your joins go smoothly.

1.  Start by identifying the variables that form the primary key in each table.
    You should usually do this based on your understanding of the data, not empirically by looking for a combination of variables that give a unique identifier.
    If you just look for variables without thinking about what they mean, you might get (un)lucky and find a combination that's unique in your current data but the relationship might not be true in general.

    For example, the altitude and longitude uniquely identify each airport, but they are not good identifiers!


In [26]:
airports %>% count(alt, lon) %>% filter(n > 1)

alt,lon,n
<dbl>,<dbl>,<int>



2.  Check that none of the variables in the primary key are missing.
    If a value is missing then it can't identify an observation!

3.  Check that your foreign keys match primary keys in another table.
    The best way to do this is with an `anti_join()`.
    It's common for keys not to match because of data entry errors.
    Fixing these is often a lot of work.

    If you do have missing keys, you'll need to be thoughtful about your use of inner vs. outer joins, carefully considering whether or not you want to drop rows that don't have a match.

Be aware that simply checking the number of rows before and after the join is not sufficient to ensure that your join has gone smoothly.
If you have an inner join with duplicate keys in both tables, you might get unlucky as the number of dropped rows might exactly equal the number of duplicated rows!

## Set operations 

The final type of two-table verb are the set operations.
Generally, I use these the least frequently, but they are occasionally useful when you want to break a single complex filter into simpler pieces.
All these operations work with a complete row, comparing the values of every variable.
These expect the `x` and `y` inputs to have the same variables, and treat the observations like sets:

-   `intersect(x, y)`: return only observations in both `x` and `y`.
-   `union(x, y)`: return unique observations in `x` and `y`.
-   `setdiff(x, y)`: return observations in `x`, but not in `y`.

Given this simple data:



In [27]:
df1 <- tribble(
  ~x, ~y,
   1,  1,
   2,  1
)
df2 <- tribble(
  ~x, ~y,
   1,  1,
   1,  2
)


The four possibilities are:


In [28]:
intersect(df1, df2)

x,y
<dbl>,<dbl>
1,1


In [29]:
# Note that we get 3 rows, not 4
union(df1, df2)

x,y
<dbl>,<dbl>
1,1
2,1
1,2


In [30]:
setdiff(df1, df2)

x,y
<dbl>,<dbl>
2,1


In [31]:
setdiff(df2, df1)

x,y
<dbl>,<dbl>
1,2
